In [1]:
import numpy as np
import pandas as pd

In [2]:
raw_data = pd.read_csv("data/raw_data.csv")

In [ ]:
columns = ['sid', 'trial', 'network', 'stage', 'who', 'color', 'degree', 'RD', 'action']
dfs = []
for sid in raw_data['SID'].unique():
    print(sid)
    for trial in raw_data.query('SID==@sid')['game_order'].unique():
        trialdata = raw_data.query('SID==@sid & game_order==@trial')
        network = trialdata['game_index'].unique()[0]
        for stage in range(4):
            if stage==0:
                color = trialdata.query("stage==0 & event_type==2")['color'].to_numpy()[0]
                degree = trialdata.query("stage==1 & event_type==5")['degree'].to_numpy()[0]
                RD = 0
                who = trialdata.query("stage==1 & event_type==5")['observed_node'].to_numpy()[0]
                act = trialdata.query("stage==0 & event_type==4")['color'].to_numpy()[0]
                df = pd.DataFrame([[sid, trial, network, stage, who, color, degree, RD, act]], columns=columns)
                dfs.append(df)
            else:
                stagedata = trialdata.query("stage==@stage")
                whos = stagedata.query("event_type==6")['observed_node'].to_numpy()
                for who in whos:
                    whodata = stagedata.query("observed_node==@who & event_type==6")
                    color = whodata['color'].to_numpy()[0]
                    degree = whodata['degree'].to_numpy()[0]
                    RD = whodata["neighbors' RD"].to_numpy()[0]
                    act = stagedata.query("event_type==4")['color'].unique()[0]
                    df = pd.DataFrame([[sid, trial, network, stage, who, color, degree, RD, act]], columns=columns)
                    dfs.append(df)
        
data = pd.concat(dfs, ignore_index=True)
data.to_pickle("data/behavior.pkl")

In [61]:
data

,sid,trial,network,stage,who,color,degree,RD,action
0,1,1,9,0,2,1,3,0.00000,1
1,1,1,9,1,6,0,3,0.23077,1
2,1,1,9,1,5,0,3,0.23077,1
3,1,1,9,1,7,1,4,0.30769,1
4,1,1,9,2,5,0,3,0.23077,0
...,...,...,...,...,...,...,...,...,...
86780,224,40,9,1,6,0,3,0.33333,0
86781,224,40,9,2,6,0,3,0.33333,1
86782,224,40,9,2,7,1,4,0.44444,1
86783,224,40,9,3,7,1,4,0.44444,1
